In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

import pandas as pd

# 读取CSV文件
raw_dataset = pd.read_csv('dataset_phishing.csv')

raw_dataset.head()

pd.set_option('display.max_rows', 500)
raw_dataset.isna().sum()
# no missing values

pd.reset_option('display.max_rows')

original_dataset = raw_dataset.copy()


class_map = {'legitimate':0, 'phishing':1}
original_dataset['status'] = original_dataset['status'].map(class_map)

# Drop the 'url' column
numeric_dataset = original_dataset.drop(columns=['url'])

# Compute the correlation matrix
corr_matrix = numeric_dataset.corr()
corr_matrix['status']
#same as corr_matrix.iloc[[87]].T
# last column and last row are the same
status_corr = corr_matrix['status']
status_corr.shape

def feature_selector_correlation(cmatrix, threshold):
    
    selected_features = []
    feature_score = []
    i=0
    for score in cmatrix:
        if abs(score)>threshold:
            selected_features.append(cmatrix.index[i])
            feature_score.append( ['{:3f}'.format(score)])
        i+=1
    result = list(zip(selected_features,feature_score)) 
    return result


features_selected = feature_selector_correlation(status_corr, 0.2)
features_selected

selected_features = [i for (i,j) in features_selected if i != 'status']
selected_features

X_selected = original_dataset[selected_features]
X_selected

X_selected.shape

y = original_dataset['status']
y


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_selected, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle = True)

from sklearn.ensemble import RandomForestClassifier
model_random_forest = RandomForestClassifier(n_estimators=350,
                                             random_state=42,
                                            )
model_random_forest.fit(X_train,y_train)

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

def custom_accuracy_set (model, X_train, X_test, y_train, y_test, train=True):
    
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    
    
    if train:
        x = X_train
        y = y_train
    elif not train:
        x = X_test
        y = y_test
        
    y_predicted = model.predict(x)
    
    accuracy = accuracy_score(y, y_predicted)
    print('model accuracy: {0:4f}'.format(accuracy))


# train accuracy

custom_accuracy_set(model_random_forest, X_train, X_test, y_train, y_test, train=True)

# test accuracy

custom_accuracy_set(model_random_forest, X_train, X_test, y_train, y_test, train=False)

In [1]:
import re
from urllib.parse import urlparse, urlsplit

# Assume 'url' is the URL we're analyzing
parsed_url = urlparse(url)
extracted_url = urlsplit(url)

features = {
    'length_url': len(url),
    'length_hostname': len(parsed_url.netloc),
    'ip': 1 if re.match(r"\b(?:\d{1,3}\.){3}\d{1,3}\b", parsed_url.netloc) else 0,
    'nb_dots': url.count('.'),
    'nb_hyphens': url.count('-'),
    'nb_at': url.count('@'),
    'nb_qm': url.count('?'),
    'nb_and': url.count('&'),
    'nb_or': url.count('|'),  # Assuming '|' is meant by 'or'
    'nb_eq': url.count('='),
    'nb_underscore': url.count('_'),
    'nb_tilde': url.count('~'),
    'nb_percent': url.count('%'),
    'nb_slash': url.count('/'),
    'b_star': 1 if '*' in url else 0,
    'nb_colon': url.count(':'),
    'nb_comma': url.count(','),
    'nb_semicolumn': url.count(';'),
    'nb_dollar': url.count('$'),
    'nb_space': url.count(' '),
    'nb_www': 1 if 'www' in parsed_url.netloc else 0,
    'nb_com': 1 if '.com' in parsed_url.netloc else 0,
    'nb_dslash':
    'http_in_path': 1 if 'http' in parsed_url.path else 0, # Assuming this checks for 'http' in the path
    'https_token': 1 if parsed_url.scheme == 'https' else 0,
    # 'ratio_digits_url': ... # This would need to be calculated as ratio of digits to length of URL
    # def calculate_ratio_digits_url(url):
    # digit_count = sum(c.isdigit() for c in url)
    # total_characters = len(url)
    # ratio = digit_count / total_characters if total_characters > 0 else 0
    # return ratio
    # # Example URL
    # url = 'https://www.example123.com/page?id=45'
    # # Calculate the ratio
    # ratio = calculate_ratio_digits_url(url)
    # print("Ratio of digits in URL:", ratio)

    # 'ratio_digits_host': ... 
    'punycode': 1 if 'xn--' in parsed_url.netloc else 0,
    'port': parsed_url.port if parsed_url.port else 0, # Assuming default ports are not to be considered
    'tld_in_path': 1 if re.search(r'\.' + re.escape(extracted_url.suffix) + r'(?=/|$)', parsed_url.path) else 0,
    'tld_in_subdomain': 1 if re.search(r'\.' + re.escape(extracted_url.suffix), extracted_url.subdomain) else 0,
    'abnormal_subdomain': 1 if 'www' not in extracted_url.subdomain and extracted_url.subdomain else 0,
    'nb_subdomains': len(extracted_url.subdomain.split('.')) if extracted_url.subdomain else 0,
    # 'prefix_suffix': ... # This needs to be defined, perhaps as presence of '-' in the domain
    'random_domain': 1 if re.search(r"(?=.{10,})(?:[a-z0-9]*[a-z][a-z0-9]*){3,}\.(?:com|net|org|info|biz)$", extracted_url.domain) is None else 0,
    # ... continue mapping other features ...
}

# You would need to write your logic to calculate the rest of the features, like 'ratio_digits_url' etc.


SyntaxError: incomplete input (3129381997.py, line 39)

In [2]:
def calculate_ratio_digits_url(url):
    digit_count = sum(c.isdigit() for c in url)
    total_characters = len(url)
    ratio = digit_count / total_characters if total_characters > 0 else 0
    return ratio

# Example URL
url = 'https://www.example123.com/page?id=45'

# Calculate the ratio
ratio = calculate_ratio_digits_url(url)
print("Ratio of digits in URL:", ratio)


Ratio of digits in URL: 0.13513513513513514


In [3]:
from urllib.parse import urlparse

def calculate_ratio_digits_host(url):
    # Parse the URL to get the host
    host = urlparse(url).netloc
    
    # Count digits in the host
    digit_count = sum(c.isdigit() for c in host)
    
    # Total number of characters in the host
    total_characters = len(host)
    
    # Calculate the ratio
    ratio = digit_count / total_characters if total_characters > 0 else 0
    
    return ratio

# Example URL
url = 'http://www.2345daohang.com/'

# Calculate the ratio
ratio = calculate_ratio_digits_host(url)
print("Ratio of digits in host:", ratio)

Ratio of digits in host: 0.21052631578947367
